In [1]:
import os
import cv2
import random
import shutil
import time
import matplotlib
import glob
import operator
import psutil
import numpy as np
import math
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from shutil import copyfile
import pandas as pd
import PIL
from mlxtend.plotting import plot_confusion_matrix

In [2]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
classify_train_dir = str(input("Classify Train Dir: "))
classify_train = os.path.join(classify_train_dir, 'classify train')

    
TRAINING_DIR = os.path.join(classify_train, 'training')
VALIDATION_DIR = os.path.join(classify_train, 'validation')
TESTING_DIR = os.path.join(classify_train, 'testing')

Classify Train Dir: /mnt/335fff25-d210-4fe2-8249-4ba8b4a17ce4/DATASETS/Shrabana/cov-cxr


In [5]:
TRAINING_DIR, VALIDATION_DIR, TESTING_DIR

('/mnt/335fff25-d210-4fe2-8249-4ba8b4a17ce4/DATASETS/Shrabana/cov-cxr/classify train/training',
 '/mnt/335fff25-d210-4fe2-8249-4ba8b4a17ce4/DATASETS/Shrabana/cov-cxr/classify train/validation',
 '/mnt/335fff25-d210-4fe2-8249-4ba8b4a17ce4/DATASETS/Shrabana/cov-cxr/classify train/testing')

In [6]:
learning_rate = float(input("Enter the initial learning rate: "))
epoch = int(input("Enter the maximum number of epochs: "))
batch_size = int(input("Enter batch size: "))

Enter the initial learning rate: 0.001
Enter the maximum number of epochs: 100
Enter batch size: 8


In [7]:
char_name = str(input("Enter name of the characteristics folder: "))

char = os.path.join(classify_train_dir, char_name)

if not os.path.exists(char):
    os.mkdir(char)
else:
    replace = str(input("Folder already exists ! Do you want to replace it ?(Y/N) "))
    if replace.upper() == 'Y':      
        shutil.rmtree(char)
        os.mkdir(char)
    elif replace.upper() == 'N':
        pass

Enter name of the characteristics folder: vgg16
Folder already exists ! Do you want to replace it ?(Y/N) y


In [8]:
def lr_schedule(epoch):
    return learning_rate * (0.1 ** int(epoch / 10))
    
best_model_address = os.path.join(char, 'best_model.h5')

In [9]:
monitor = int(input("Press 1 to monitor Validation Accuracy\nPress 2 to monitor Validation Loss\nPress 3 to monitor Training Accuracy\nPress 4 to monitor Training Loss\n"))
patience = int(input('Enter number of epochs that will produce monitored quantity with no improvement after which training will be stopped: '))


if monitor == 1:
    metric = 'val_accuracy'
    mode = 'max'
    print("\nMONITORING VALIDATION ACCURACY..........\n")

elif monitor == 2:
    metric = 'val_loss'
    mode = 'min'
    print("\nMONITORING VALIDATION LOSS..........\n")

elif monitor == 3:
    metric = 'accuracy'
    mode = 'max'
    print("\nMONITORING TRAINING ACCURACY..........\n")

elif monitor == 4:
    metric = 'loss'
    mode = 'min'
    print("\nMONITORING TRAINING LOSS..........\n")

callback = [keras.callbacks.LearningRateScheduler(lr_schedule, verbose = 1),
            keras.callbacks.EarlyStopping(monitor = metric, min_delta = 0.001, patience = patience, verbose=1, mode = mode, restore_best_weights = True),
            keras.callbacks.ModelCheckpoint(best_model_address, monitor = metric, verbose=1, save_best_only=True, save_weights_only=False, mode = mode , period=1)]

print("\nTraining will stop if Validation Accuracy doesn't show any improvements for " + str(patience) + " epcohs\n")

Press 1 to monitor Validation Accuracy
Press 2 to monitor Validation Loss
Press 3 to monitor Training Accuracy
Press 4 to monitor Training Loss
4
Enter number of epochs that will produce monitored quantity with no improvement after which training will be stopped: 10

MONITORING TRAINING LOSS..........


Training will stop if Validation Accuracy doesn't show any improvements for 10 epcohs



In [10]:
def vgg16():
    print("\nTRAINING ON VGG16 MODEL:-")

    base_model = keras.applications.vgg16.VGG16(input_shape = dim, weights = 'imagenet', include_top = False)

    x = base_model.output
    x = GlobalMaxPooling2D()(x)
    x = Dense(dense)(x)
    x = LeakyReLU()(x)
    x = Dropout(dropout)(x)
    x = BatchNormalization()(x)

    predictions = Dense(output_layer, activation = output_activation)(x)

    model = Model(inputs = base_model.input, outputs=predictions)

    train_base_model = str(input("Do you want to train the base model of VGG16?(Y/N) "))
    if train_base_model.upper() == 'Y':
        for layer in base_model.layers:
            layer.trainable = True
    elif train_base_model.upper() == 'N':
        for layer in base_model.layers:
            layer.trainable = False

    return model

In [11]:
class_no = len(os.listdir(TRAINING_DIR))

if class_no > 2:
    print("This is a " + str(class_no) + "-Class Classification")
    output_activation = 'softmax'
    losses = 'categorical_crossentropy'
    class_mode = 'categorical'
    output_layer = class_no
else:
    print("This is a Binary Classification")
    output_activation = 'sigmoid'
    losses = 'binary_crossentropy'
    class_mode = 'binary'
    output_layer = 1

This is a 3-Class Classification


In [12]:
def optimizer_selection():
    print("\nSelect a optimizer which will reduce the loss of the model.\n")

    optimizer_select = int(input("Press 1 to select Stochastic Gradient Descent\nPress 2 to select RMSprop\nPress 3 to select Adagrad\nPress 4 to select Adadelta\nPress 5 to select Adam\nPress 6 to select Adamax\nPress 7 to select Nadam\n"))

    if optimizer_select == 1:
        optimizer = SGD(lr = learning_rate, decay = 1e-6, momentum = 0.9, nesterov = True)

    elif optimizer_select == 2:
        optimizer = RMSprop(learning_rate, rho = 0.9)

    elif optimizer_select == 3:
        optimizer = Adagrad(learning_rate)

    elif optimizer_select == 4:
        optimizer = Adadelta(learning_rate, rho = 0.95)

    elif optimizer_select == 5:
        optimizer = Adam(learning_rate = learning_rate, beta_1 = 0.9, beta_2 = 0.999, amsgrad = False)

    elif optimizer_select == 6:
        optimizer = Adamax(learning_rate = learning_rate, beta_1 = 0.9, beta_2 = 0.999)

    elif optimizer_select == 7:
        optimizer = Nadam(learning_rate = learning_rate, beta_1 = 0.9, beta_2 = 0.999)
   
    return optimizer

In [13]:
h = int(input("Image Dimension(H or W): "))
w = h
dim = [h,w,3]

Image Dimension(H or W): 128


In [14]:
train_datagen = ImageDataGenerator(rescale=1.0/255.)
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size = batch_size,
                                                    class_mode = class_mode,
                                                    target_size = (h,w),
                                                    shuffle=True)

validation_datagen = ImageDataGenerator(rescale=1.0/255.)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size = batch_size,
                                                              class_mode = class_mode,
                                                              target_size = (h,w),
                                                              shuffle=True)

test_datagen = ImageDataGenerator(rescale=1.0/255.)
test_generator = test_datagen.flow_from_directory(TESTING_DIR,
                                                  batch_size = batch_size,
                                                  class_mode = class_mode,
                                                  target_size = (h,w),
                                                  shuffle=True)

Found 8591 images belonging to 3 classes.
Found 399 images belonging to 3 classes.
Found 1995 images belonging to 3 classes.


In [15]:
dense = int(input("Enter the no. of neurons in dense layer: "))
dropout = float(input("Enter the dropout percentage: "))
dropout = dropout/100

optimizer = optimizer_selection()

Enter the no. of neurons in dense layer: 128
Enter the dropout percentage: 20

Select a optimizer which will reduce the loss of the model.

Press 1 to select Stochastic Gradient Descent
Press 2 to select RMSprop
Press 3 to select Adagrad
Press 4 to select Adadelta
Press 5 to select Adam
Press 6 to select Adamax
Press 7 to select Nadam
5


In [16]:
model= vgg16()
model.compile(optimizer = optimizer, loss = losses, metrics = ['accuracy', 
                                                               tf.keras.metrics.Precision(), 
                                                               tf.keras.metrics.Recall(), 
                                                               tf.keras.metrics.TruePositives(), 
                                                               tf.keras.metrics.TrueNegatives(), 
                                                               tf.keras.metrics.FalsePositives(),
                                                               tf.keras.metrics.FalseNegatives()])


model.summary()


TRAINING ON VGG16 MODEL:-
Do you want to train the base model of VGG16?(Y/N) y
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_______________________________________________

In [ ]:
with tf.device('/GPU:0'):
    start = time.time()
    history = model.fit(train_generator,
                        epochs = epoch,
                        verbose = 1,
                        callbacks = callback,
                        validation_data = validation_generator,
                        shuffle=True)

    end = time.time()
    duration = end-start

train_score = model.evaluate(train_generator)
val_score = model.evaluate(validation_generator)
test_score = model.evaluate(test_generator)

Epoch 1/100

Epoch 00001: LearningRateScheduler reducing learning rate to 0.001.
1074/1074 [==============================] - 62s 53ms/step - loss: 0.9333 - accuracy: 0.6090 - precision: 0.6291 - recall: 0.5679 - true_positives: 2720.1656 - true_negatives: 7385.2428 - false_positives: 1222.1674 - false_negatives: 1583.5395 - val_loss: 0.2628 - val_accuracy: 0.9223 - val_precision: 0.9244 - val_recall: 0.9198 - val_true_positives: 367.0000 - val_true_negatives: 768.0000 - val_false_positives: 30.0000 - val_false_negatives: 32.0000

Epoch 00001: loss improved from inf to 0.65613, saving model to /mnt/335fff25-d210-4fe2-8249-4ba8b4a17ce4/DATASETS/Shrabana/cov-cxr/vgg16/best_model.h5
Epoch 2/100

Epoch 00002: LearningRateScheduler reducing learning rate to 0.001.
1074/1074 [==============================] - 57s 53ms/step - loss: 0.3810 - accuracy: 0.8588 - precision: 0.8674 - recall: 0.8490 - true_positives: 3677.0884 - true_negatives: 8067.1516 - false_positives: 538.8893 - false_negative

Epoch 13/100

Epoch 00013: LearningRateScheduler reducing learning rate to 0.0001.
1074/1074 [==============================] - 57s 53ms/step - loss: 0.0875 - accuracy: 0.9745 - precision: 0.9761 - recall: 0.9731 - true_positives: 4190.5981 - true_negatives: 8505.7740 - false_positives: 101.0707 - false_negatives: 112.8242 - val_loss: 0.0501 - val_accuracy: 0.9825 - val_precision: 0.9825 - val_recall: 0.9825 - val_true_positives: 392.0000 - val_true_negatives: 791.0000 - val_false_positives: 7.0000 - val_false_negatives: 7.0000

Epoch 00013: loss improved from 0.09420 to 0.08373, saving model to /mnt/335fff25-d210-4fe2-8249-4ba8b4a17ce4/DATASETS/Shrabana/cov-cxr/vgg16/best_model.h5
Epoch 14/100

Epoch 00014: LearningRateScheduler reducing learning rate to 0.0001.
1074/1074 [==============================] - 53s 49ms/step - loss: 0.0711 - accuracy: 0.9773 - precision: 0.9785 - recall: 0.9766 - true_positives: 4203.7860 - true_negatives: 8516.1777 - false_positives: 90.9628 - false_negat

1074/1074 [==============================] - 53s 49ms/step - loss: 0.0270 - accuracy: 0.9927 - precision: 0.9929 - recall: 0.9925 - true_positives: 4269.8502 - true_negatives: 8575.2884 - false_positives: 31.4595 - false_negatives: 33.5237 - val_loss: 0.0520 - val_accuracy: 0.9799 - val_precision: 0.9799 - val_recall: 0.9799 - val_true_positives: 391.0000 - val_true_negatives: 790.0000 - val_false_positives: 8.0000 - val_false_negatives: 8.0000

Epoch 00025: loss improved from 0.02855 to 0.02790, saving model to /mnt/335fff25-d210-4fe2-8249-4ba8b4a17ce4/DATASETS/Shrabana/cov-cxr/vgg16/best_model.h5
Epoch 26/100

Epoch 00026: LearningRateScheduler reducing learning rate to 1.0000000000000003e-05.
1074/1074 [==============================] - 52s 49ms/step - loss: 0.0317 - accuracy: 0.9911 - precision: 0.9913 - recall: 0.9909 - true_positives: 4260.8474 - true_negatives: 8566.2195 - false_positives: 39.9033 - false_negatives: 42.2140 - val_loss: 0.0523 - val_accuracy: 0.9799 - val_precisi

1074/1074 [==============================] - 53s 50ms/step - loss: 0.0280 - accuracy: 0.9928 - precision: 0.9937 - recall: 0.9923 - true_positives: 4270.3265 - true_negatives: 8579.4270 - false_positives: 27.5051 - false_negatives: 33.1395 - val_loss: 0.0553 - val_accuracy: 0.9749 - val_precision: 0.9749 - val_recall: 0.9749 - val_true_positives: 389.0000 - val_true_negatives: 788.0000 - val_false_positives: 10.0000 - val_false_negatives: 10.0000

Epoch 00038: loss did not improve from 0.02483
Epoch 39/100

Epoch 00039: LearningRateScheduler reducing learning rate to 1.0000000000000002e-06.
1074/1074 [==============================] - 53s 49ms/step - loss: 0.0226 - accuracy: 0.9931 - precision: 0.9934 - recall: 0.9927 - true_positives: 4272.7302 - true_negatives: 8579.1563 - false_positives: 27.5898 - false_negatives: 30.6428 - val_loss: 0.0533 - val_accuracy: 0.9749 - val_precision: 0.9774 - val_recall: 0.9749 - val_true_positives: 389.0000 - val_true_negatives: 789.0000 - val_false_p

In [ ]:
print("Execution Time: {} seconds".format(duration))

In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(len(acc))

plt.plot(epochs, acc, 'r', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title('Training and validation accuracy vs Epochs')
plt.legend()
accuracy_fig_name = "accuracy.eps"
plt.savefig(os.path.join(char, accuracy_fig_name))
plt.show()


plt.plot(epochs, loss, 'r', label="Training Loss")
plt.plot(epochs, val_loss, 'b', label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title('Training and validation loss vs Epochs')
plt.legend()
loss_fig_name = "loss.eps"

plt.savefig(os.path.join(char, loss_fig_name))
plt.show()

In [ ]:
training_accuracy = train_score[1]*100
validation_accuracy = val_score[1]*100
test_accuracy = test_score[1]*100

print("The training accuracy is: " + str(training_accuracy) + ' %')
print("The validation accuracy is: " + str(validation_accuracy) + ' %')
print("The test accuracy is: " + str(test_accuracy) + ' %')

In [ ]:
test_accuracy = test_score[1]*100
test_precision = test_score[2]*100
test_recall = test_score[3]*100
tp = int(test_score[4])
tn = int(test_score[5])
fp = int(test_score[6])
fn = int(test_score[7])

f1 = 2*((test_precision*test_recall)/(test_precision+test_recall))
sensitivity = (tp/(tp+fn))*100
specificity = (tn/(tn+fp))*100

In [ ]:
print("Test Accuracy: {}".format(test_accuracy))
print("Test Precision: {}".format(test_precision))
print("Test Recall: {}".format(test_recall))
print("True Positive: {}".format(tp))
print("Test Negetive: {}".format(tn))
print("False Positive: {}".format(fp))
print("False Negetive: {}".format(fn))
print("Sensitivity: {}".format(sensitivity))
print("Specificity: {}".format(specificity))

In [ ]:
from contextlib import redirect_stdout

readme_name_text = "readme.txt"
print("Please read the text file named " + readme_name_text + " for detailed information of the model.")

completeName_txt = os.path.join(char, readme_name_text) 

readme = open(completeName_txt, "w")

if len(os.listdir(TRAINING_DIR)) > 2:
    readme.write("This is a MULTICLASS CLASSIFICATION")
else:
    readme.write("This is a BINARY CLASSIFICATION")


readme.write("\n\n--HYPERPARAMETERS--\n")
readme.write(str(augmentation))
readme.write("\nInitial Learning Rate = " + str(learning_rate))
readme.write("\nNo. of epochs = " + str(len(acc)))
readme.write("\nBatch Size = " + str(batch_size))


readme.write("\n\n--MODEL-PARAMETERS--")
readme.write("\nActivation Function = relu")
readme.write("\nDropout = " + str(int(dropout*100)) + "%")
readme.write("\nActivation function of the output layer = " + str(output_activation))
readme.write("\nCost function of the model = " + str(losses))
readme.write("\nOptimizer = " + str(optimizer) + "\n\n")


readme.write("Trained on a VGG16 Model\n")
with redirect_stdout(readme):
    model.summary()
        
    
readme.write("\n\n--MODEL-PERFORMANCE--")
readme.write("\nTest Accuracy = " + str(test_accuracy) + " %")
readme.write("\nTest Precision = " + str(test_precision) + " %")
readme.write("\nTest Recall = " + str(test_recall) + " %")
readme.write("\nTrue Positive = " + str(tp))
readme.write("\nTrue Negetive = " + str(tn))
readme.write("\nFalse Positive = " + str(fp))
readme.write("\nFalse Negetive = " + str(fn))
readme.write("\nSensitivity = " + str(sensitivity))
readme.write("\nSpecificity = " + str(specificity) + " \n\n\n")


readme.write("\n\n--MODEL-CHARACTERISTICS--")
readme.write("\nacc = " + str(acc))
readme.write("\n\nval_acc = " + str(val_acc))
readme.write("\n\nloss = " + str(loss))
readme.write("\n\nval_loss = " + str(val_loss))


readme.write("\nExecution Time: {} seconds".format(duration))

readme.write("\n\nCreated using Self-Regulated Image Classifier using Convolution Neural Network")

readme.close()